In [1]:
import os 
import folium
from folium import plugins
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et
import pandas as pd

# # Import data from EarthPy
# data = et.data.get_data('colorado-flood')

# # Set working directory to earth-analytics
# os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [17]:
map_instruct = pd.read_csv('nursing_home_app/nhs_descriptors.csv')

In [18]:
map_instruct['geolocation_y'] = map_instruct['geolocation_y'].astype(str)

In [19]:
map_instruct['geolocation_y'].dtype

dtype('O')

In [20]:
test = map_instruct

In [21]:
NY = test[test['provider_state']=='NY']

In [22]:
map_instruct[['latitude', 'longitude']] = map_instruct.geolocation_y.str.split(expand=True)

In [23]:
map_instruct.head(1)

,Unnamed: 0,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,full_address,geolocation_y,latitude,longitude
0,0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,"HIGHWAY 191 & HOSPITAL ROAD, CHINLE, AZ, 86503","[36.16763243080752, -109.68806973451998]","[36.16763243080752,",-109.68806973451998]


In [24]:
def strip_col(input):
    input = input.strip('[,]')
    input = float(input)
    return input

In [28]:
map_instruct[map_instruct['provider_name']=='VILLAGE AT HILLSIDE']

,Unnamed: 0,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,full_address,geolocation_y,latitude,longitude
23,23,385269,VILLAGE AT HILLSIDE,400 NW HILLSIDE PARK WAY,MCMINNVILLE,OR,97128,Yamhill,"400 NW HILLSIDE PARK WAY, MCMINNVILLE, OR, 97128","[16.13238508058608, -61.58205104299407]",16.132385,-61.582051


In [26]:
map_instruct['latitude'] = map_instruct['latitude'].astype(str)
map_instruct['longitude'] = map_instruct['longitude'].astype(str)

In [27]:
map_instruct['latitude'] = map_instruct['latitude'].apply(strip_col)
map_instruct['longitude'] = map_instruct['longitude'].apply(strip_col)

In [13]:
a = folium.Map(location=[37.09024, -95.712891], 
               width='100%', 
               height='100%', 
               left='0%', 
               top='0%', 
               position='relative', 
               tiles='https://api.mapbox.com/styles/v1/jenfaith/ckm3ufeuf0v8y17qqi8d7gfre/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiamVuZmFpdGgiLCJhIjoiY2ttM3QzYnI1MDhzZTJucWRubXlzMHphdCJ9.3FsZLNb_YZlAwssJDfV4Cw', 
               attr='Map Box Attribute', 
               min_zoom=4, 
               max_zoom=None, 
               zoom_start=4,
               min_lat= -90, 
               max_lat=90, 
               min_lon=- 180, 
               max_lon=180, max_bounds=False, 
               crs='EPSG3857', control_scale=False, 
               prefer_canvas=False, no_touch=False, 
               disable_3d=False, png_enabled=False, 
               zoom_control=True)
a

In [15]:
folium.Marker(
    location=[40.76879168614267, -73.82285349073014], # coordinates for the marker
    popup='<a href="https://www.google.com/" target="_blank">THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE</a>', # pop-up label for the marker
    icon=folium.Icon(color='red', icon='home', prefix='fa')
).add_to(a)

a
#a.save('index.html')

In [16]:
folium.Marker(
    location=[40.76879168614267, -73.82285349073014], # coordinates for the marker
    popup='<a href= {href_tag} target="_blank">{name}</a>'.format(href_tag = '/report?facility_name='+'THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE'.replace(' ', '+'), name = 'THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE'), # pop-up label for the marker
    # popup='<a href="myurl" target="_blank">mytext</a>',
    icon=folium.Icon(color='red', icon='home', prefix='fa')
    ).add_to(a)

a
a.save('index.html')

In [ ]:
# Testing idea to make sure it works

facility_name = 'THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE'
html_response = '/report?facility_name='+facility_name.replace(' ', '+')

get = 'https://data.cms.gov/resource/s2uc-8wxp.json?provider_name={facility_name}'.format(facility_name = facility_name)
response = requests.get(get)

In [ ]:
# Testing idea again

facility_name = 'THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE'
html_response = '/report?facility_name='+facility_name.replace(' ', '+')

get = 'https://data.cms.gov/resource/s2uc-8wxp.json?provider_name={facility_name}'.format(facility_name = facility_name)
response = requests.get(get)

In [ ]:
tiles='https://api.mapbox.com/v4/mapbox.streets/{z}/{x}/{y}.png?access_token=mytoken',

In [ ]:
facility_name = 'THE CITADEL REHAB AND NURSING CTR AT KINGSBRIDGE'
html_response = '/report?facility_name='+facility_name.replace(' ', '+')

get = 'https://data.cms.gov/resource/s2uc-8wxp.json?provider_name={facility_name}'.format(facility_name = facility_name)
response = requests.get(get)

## Function To Generate All Points

In [125]:
data = [['CHINLE NURSING HOME', 36.16763243080752, -109.68806973451998], 
        ['PARK TERRACE CARE CENTER', 40.748088772582804, -73.86062360133637], 
        ['REGO PARK NURSING HOME', 40.748088772582804, -73.86062360133637]] 
df = pd.DataFrame(data, columns = ['provider_name', 'latitude', 'longitude'])
df

,provider_name,latitude,longitude
0,CHINLE NURSING HOME,36.167632,-109.688070
1,PARK TERRACE CARE CENTER,40.748089,-73.860624
2,REGO PARK NURSING HOME,40.748089,-73.860624


In [132]:
def generate_points(df):
    a = folium.Map(location=[37.09024, -95.712891], 
               width='100%', 
               height='100%', 
               left='0%', 
               top='0%', 
               position='relative', 
               tiles='https://api.mapbox.com/styles/v1/jenfaith/ckm3ufeuf0v8y17qqi8d7gfre/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiamVuZmFpdGgiLCJhIjoiY2ttM3QzYnI1MDhzZTJucWRubXlzMHphdCJ9.3FsZLNb_YZlAwssJDfV4Cw', 
               attr='Map Box Attribute', 
               min_zoom=4, 
               max_zoom=None, 
               zoom_start=4,
               min_lat= -90, 
               max_lat=90, 
               min_lon=- 180, 
               max_lon=180, max_bounds=False, 
               crs='EPSG3857', control_scale=False, 
               prefer_canvas=False, no_touch=False, 
               disable_3d=False, png_enabled=False, 
               zoom_control=True)
    
    for i in range(len(df)):
        name = df['provider_name'].iloc[i]
        i = folium.Marker(
            location=[df['latitude'].iloc[i], df['longitude'].iloc[i]], # coordinates for the marker
            popup='<a href= {href_tag} target="_blank">{name}</a>'.format(href_tag = 
                                                                          '/report?facility_name='
                                                                          + name.replace(' ', '+'), 
                                                                          name = name), # pop-up label for the marker
            icon=folium.Icon(color='red', icon='home', prefix='fa')
        ).add_to(a)
    return a
    

In [107]:
href_tag = '/report?facility_name='+ name.replace(' ', '+')
href_tag

'/report?facility_name=THE+CITADEL+REHAB+AND+NURSING+CTR+AT+KINGSBRIDGE'

In [ ]:
/report?facility_name=MEDICINE+WHEEL+VILLAGE

In [133]:
new_map = generate_points(df)

In [130]:
new_map.save('index.html')

In [134]:
new_map

In [191]:
map_final = generate_points(map_instruct)

In [193]:
map_final.save('index.html')